# 6. Mitochondrial genome annotation

## 6.1 Exctract mitobim results
If you are using my pre-computed assemblies

In [ ]:
#import os
#from misc import execute_cline

#for key in smpl2ref:
#    path = '%s_mitobim'%key
#    for root, dirs, files in os.walk(path):
#        for f in files:
#            cline = 'gzip -d %s/%s'%(root,f)
#            execute_cline(cline)

## 6.2 Make consensus fasta files
and put links to all of them in one directory, together with links to ace alignments

In [1]:
smpl2ref = {
    'MentL30'    :['NC_024097.1',15],
    'MareHarA'   :['NC_026554.1',4],
    'MjavVW4'    :['NC_026556.1',1],
    'MjavVW5'    :['NC_026556.1',1],
    'MareL28'    :['NC_026554.1',1],
    'MincA14'    :['NC_024097.1',1],
    'MincW1'    :['NC_024097.1',1],
    'MjavLD17'   :['NC_026556.1',1],
    'MincL27'    :['NC_024097.1',1],
    'MjavLD15'   :['NC_026556.1',1],
    'MjavL57'    :['NC_026556.1',1],
    'MincVW6'    :['NC_024097.1',1],
    'MincL9'     :['NC_024097.1',1],
    'Minc557R'   :['NC_024097.1',1],
    'MincL19'    :['NC_024097.1',1],
    'MareL32'    :['NC_026554.1',4],
    'MincHarC'   :['NC_024097.1',1],
    'MfloSJF1'   :['NC_024097.1',15],
    'MfloJB5'    :['NC_024097.1',6],
    'MlanSJH1'   :['NC_024097.1',6]
}

In [2]:
from glob import glob
from misc import makedir, execute_cline
from os.path import exists, abspath

makedir('mitochondrial_assemblies')
maf_template = '/{smpl}-{ref}_assembly/{smpl}-{ref}_d_results/{smpl}-{ref}_out.maf'
miraconvert_cline = 'miraconvert {params} {inf} {outf}'
ln_cline = 'ln -s {inf} {outf}'

for mitobim_dir in glob('*_mitobim'):
    if not any([key in mitobim_dir for key in smpl2ref.keys()]):
        continue
    smpl = mitobim_dir.split('_')[0]
    ref = smpl2ref[smpl][0]
    iterations = list(glob(mitobim_dir + '/iteration*'))
    max_iter_dir = sorted(iterations,
                          key=lambda i:
                          int(i.replace(mitobim_dir + '/iteration','')))[-1]
    maf = max_iter_dir + maf_template.format(smpl=smpl, ref=ref)
    ace = maf.replace('maf','ace')
    fasta = maf.replace('.maf','_%s.unpadded.fasta' % smpl)
    acelink = 'mitochondrial_assemblies/'+ace.split('/')[-1]
    fastalink = 'mitochondrial_assemblies/'+fasta.split('/')[-1]
    if not exists(ace):
        out, err = execute_cline(miraconvert_cline.format(params='', inf=maf, outf=ace))
    if not exists(acelink):
        out, err = execute_cline(ln_cline.format(inf=abspath(ace), outf=acelink))

    out, err = execute_cline(miraconvert_cline.format(params='-f maf -t fasta -A "SOLEXA_SETTINGS  -CO:fnicpst=yes"',
                                                      inf=maf, outf=fasta.split('_%s'%smpl)[0]))
    
    print err
    
    if not exists(fastalink):
        out, err = execute_cline(ln_cline.format(inf=abspath(fasta), outf=fastalink))
    

misc.py:291: UserWarning: keeping existing fpath mitochondrial_assemblies
  warnings.warn('keeping existing fpath %s'%name)


## 6.3 Run exonerate
For each contig in the assembly run a single alignment of the contig with its respective bait and extract genes which are in this case also the CDSs.

In [6]:
from os import remove
from Bio import SeqIO
from StringIO import StringIO
import misc

cds_loci = ['COX1','ND1','ND2','COX3','ND6','ND4L',
            'COX2','ND3','CYTB','ND4','ATP6','ND5']

## exonerate cline
# query and target
a = "exonerate -q %s -t %s -m %s -n 1 --geneticcode 5 --showtargetgff "
# search CDS in DNA and get only the best result
b = '--ryo \"STARTGENE\\n>{0}qi\\n{0}tas\\nENDGENE\\nSTARTCDS\\n>{0}qi\\n{0}tcs\\nENDCDS\\n \" '.format('%')

def iter_records(fpath):
    with open(fpath,'r') as lines:
        record = []
        get = False
        for line in lines:
            if 'C4 Alignment:' in line:
                get = True
            if get and 'C4 Alignment:' in line:
                if len(record) > 0:
                    yield record
                    record = [line]
                else:
                    record = [line]
            elif get:
                record.append(line)
        yield record

def parse_a_single_record(record):
    
    output = {}
    
    def parse_line_element(ind, parter, element_name):
        elementline = record[ind]
        try:
            output[element_name] = elementline.rstrip().split(parter)[1]
            if ' [revcomp]' in elementline:
                output[element_name] = output[element_name].replace(' [revcomp]','')
            elif ' -> ' in output[element_name] and \
                 element_name == 'Target_start':
                output[element_name] = output[element_name].split(' -> ')[0]
            elif ' -> ' in output[element_name] and \
                 element_name == 'Target_end':
                output[element_name] = output[element_name].split(' -> ')[1]
            try:
                output[element_name] = int(output[element_name])
            except:
                pass
        except:
            raise RuntimeError("%s not found"%parter)

            
    line_elements = [[2, 'Query: ', 'Query'],
                     [3, 'Target: ', 'Target'],
                     [5, 'Raw score: ', 'Score'],
                     [7, 'Target range: ', 'Target_start'],
                     [7, 'Target range: ', 'Target_end']]
    
    
    for ind, parter, element_name in line_elements:
        parse_line_element(ind, parter, element_name)
        
    def parse_block_element(init, halt, element_name):
        element = ""
        get = False
        for line in record:
            if init in line:
                get = True
            elif halt in line:
                get = False
            elif get:
                element += line
        output[element_name] = element
        
    block_elements = [['STARTGENE','ENDGENE','Gene'],
                      ['STARTCDS','ENDCDS','CDS'],
                      ['--- START OF GFF DUMP ---',
                       '--- END OF GFF DUMP ---',
                       'GFF']
                     ]
    for init, halt, element_name in block_elements:
        parse_block_element(init, halt, element_name)
        
    return output

for smpl in smpl2ref:
    ref = smpl2ref[smpl][0]
    protref = SeqIO.to_dict(SeqIO.parse('mito_references/%s_proteins.fasta'%ref,'fasta'))
    genref  = SeqIO.to_dict(SeqIO.parse('mito_references/%s_genes.fasta'%ref,'fasta'))
    assembly = \
    SeqIO.parse('mitochondrial_assemblies/{0}-{1}_out_{0}.unpadded.fasta'.format(smpl,ref),
                'fasta') 
    
    gene_file = open('mitochondrial_assemblies/%s_genes.fasta'%smpl,'wt')
    
    for r in assembly:
        gene = r.id.replace(ref+'_','').replace('_bb','')
        is_cds = False
        if gene in cds_loci:
            is_cds = True
        if is_cds:
            with open('mito_q_temp.fasta','wt') as hndl:
                hndl.write(protref[ref+'_'+gene].format('fasta'))
        else:
            with open('mito_q_temp.fasta','wt') as hndl:
                hndl.write(genref[ref+'_'+gene].format('fasta'))
        with open('mito_t_temp.fasta','wt') as hndl:
            hndl.write('>'+r.id+'\n'+str(r.seq).replace('x','n')+'\n')
        
        model = 'est2genome'
        if is_cds:
            model = 'protein2genome'
        
        thiscline = a%('mito_q_temp.fasta',
                       'mito_t_temp.fasta',
                        model) + b    
        #print thiscline
        
        out, err = misc.execute_cline(thiscline)
        #print err
        outhndl = open('mitoexo','wt')
        outhndl.write(out)
        outhndl.close()
        
        if len(open('mitoexo','r').readlines()) < 6:
            print 'no', gene, smpl
            continue
        
        results = parse_a_single_record(list(iter_records('mitoexo'))[0])
        newrecord = SeqIO.read(StringIO(results['Gene']),'fasta')
        newrecord.id = "%s_%s" % (smpl, newrecord.id)
        gene_file.write(newrecord.format('fasta'))
        
        remove('mitoexo')
        remove('mito_q_temp.fasta')
        remove('mito_t_temp.fasta')
        
    gene_file.close()